In [333]:
import pandas as pd


In [334]:
df = pd.read_excel(".././data/original/pop_landscape_finale.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 49 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   nom_prenom_match          498 non-null    object        
 1   natt                      498 non-null    int64         
 2   NEFG                      498 non-null    int64         
 3   NEFG2                     498 non-null    object        
 4   birth_date                498 non-null    datetime64[ns]
 5   LT_age                    498 non-null    float64       
 6   LT_date                   498 non-null    datetime64[ns]
 7   center                    498 non-null    object        
 8   aetiology                 498 non-null    int64         
 9   gender                    498 non-null    object        
 10  taille                    498 non-null    float64       
 11  bmi                       498 non-null    float64       
 12  donneur_age           

In [335]:
df.columns

Index(['nom_prenom_match', 'natt', 'NEFG', 'NEFG2', 'birth_date', 'LT_age',
       'LT_date', 'center', 'aetiology', 'gender', 'taille', 'bmi',
       'donneur_age', 'donneur_PF', 'donneur_tabac', 'donneur_ISHLT', 'LAS',
       'LAS_SU', 'LAS_2', 'SU', 'type_tp', 'ischemie_duree', 'ECMO_postop',
       'ECMO_postop_duree_jours', 'DPG3_72h', 'induction', 'CMV_DR', 'DSA_pre',
       'DSA_last', 'DSA_last_date', 'patient_survival_status',
       'patient_survival_ddn', 'patient_survival_delay',
       'graft_survival_status', 'graft_survival_ddn', 'graft_survival_delay',
       're_tx', 'CLAD_status', 'CLAD_ddn', 'CLAD_delay', 'LT_age_2',
       'patient_survival_delay_2', 'graft_survival_delay_2', 'CLAD_delay_2',
       'donneur_tabac_2', 'aetiology_2', 'aetiology_3', 'aetiology_3_and_1',
       'gender_2'],
      dtype='object')

In [336]:
df = df[["NEFG","NEFG2", "LT_age", "aetiology", "gender", "bmi",
    "donneur_age", "donneur_PF", "donneur_tabac","LAS","SU",
    "type_tp", "ECMO_postop", "ECMO_postop_duree_jours",
    "induction","CMV_DR", "DSA_pre", 
     'patient_survival_status', 'patient_survival_delay',
       'graft_survival_status' , 'graft_survival_delay',
       're_tx',]]

In [337]:
# Get object (categorical) columns
object_columns = df.select_dtypes(include=['object']).columns
print("Object columns:", list(object_columns))

# Get datetime columns
datetime_columns = df.select_dtypes(include=['datetime64', 'datetime64[ns]']).columns
print("Datetime columns:", list(datetime_columns))

Object columns: ['NEFG2', 'gender', 'CMV_DR']
Datetime columns: []


In [338]:
# Create a mapping dictionary
gender_map = {'female': 0, 'male': 1}

# Apply the mapping to create a new column or transform the existing one
df['gender'] = df['gender'].str.lower().map(gender_map)


In [339]:
# Standardize format without numerical mapping
df['CMV_DR'] = df['CMV_DR'].astype('category')

# If you need to get the codes later:
df['CMV_DR_coded'] = df['CMV_DR'].cat.codes


In [340]:
df_pca = pd.read_csv(".././data/analysis/pca_features.csv", sep=",")
df_pca

,NEFG2,PC1,PC2,PC3
0,119222,-6.059952,-2.337604,-0.112934
1,119430#2,-4.147300,2.131158,-0.582274
2,129042#2,-1.878337,2.039656,4.220944
3,131960,-3.480378,-0.155022,1.850744
4,140937#2,0.614241,2.497368,7.819445
...,...,...,...,...
493,394318,-1.522323,-2.423574,4.274153
494,394410,-3.869811,-6.337252,1.451607
495,394550,-6.167030,-2.297303,0.187971
496,394636,-9.042095,-6.377158,-0.285073


In [341]:
# Remove unwanted characters from 'NEFG2' in both dataframes and convert to strings
df_pca['NEFG2_cleaned'] = df_pca['NEFG2'].astype(str).str.replace('[^0-9]', '', regex=True)
df['NEFG2_cleaned'] = df['NEFG2'].astype(str).str.replace('[^0-9]', '', regex=True)

# Check if they have the same patients
common_patients = set(df_pca['NEFG2_cleaned']).intersection(set(df['NEFG2_cleaned']))
only_in_pca = set(df_pca['NEFG2_cleaned']) - set(df['NEFG2_cleaned'])
only_in_other = set(df['NEFG2_cleaned']) - set(df_pca['NEFG2_cleaned'])

set(df["NEFG2_cleaned"].unique()).difference(set(df_pca["NEFG2_cleaned"].unique()))


set()

In [342]:
merged_df = pd.merge(df_pca, df, on='NEFG2_cleaned', how='outer', suffixes=('_pca', '_pop'))

merged_df.columns

Index(['NEFG2_pca', 'PC1', 'PC2', 'PC3', 'NEFG2_cleaned', 'NEFG', 'NEFG2_pop',
       'LT_age', 'aetiology', 'gender', 'bmi', 'donneur_age', 'donneur_PF',
       'donneur_tabac', 'LAS', 'SU', 'type_tp', 'ECMO_postop',
       'ECMO_postop_duree_jours', 'induction', 'CMV_DR', 'DSA_pre',
       'patient_survival_status', 'patient_survival_delay',
       'graft_survival_status', 'graft_survival_delay', 're_tx',
       'CMV_DR_coded'],
      dtype='object')

In [349]:
merged_df["patient_survival_status"].unique()

array([0, 1])

In [343]:
merged_df.drop(columns=["NEFG2_pca", 'NEFG2_cleaned', 'NEFG','NEFG2_pop',"CMV_DR"],inplace=True)
merged_df.to_csv(".././data/analysis/df_clustering_pca.csv")

In [344]:
df_raw_features = pd.read_csv(".././data/analysis/df_new_features.csv")

In [345]:
# Remove unwanted characters from 'NEFG2' in both dataframes and convert to strings
df_raw_features['NEFG2_cleaned'] = df_raw_features['NEFG2'].astype(str).str.replace('[^0-9]', '', regex=True)
df['NEFG2_cleaned'] = df['NEFG2'].astype(str).str.replace('[^0-9]', '', regex=True)

# Check if they have the same patients
common_patients = set(df_raw_features['NEFG2_cleaned']).intersection(set(df['NEFG2_cleaned']))
only_in_pca = set(df_raw_features['NEFG2_cleaned']) - set(df['NEFG2_cleaned'])
only_in_other = set(df['NEFG2_cleaned']) - set(df_raw_features['NEFG2_cleaned'])

set(df["NEFG2_cleaned"].unique()).difference(set(df_raw_features["NEFG2_cleaned"].unique()))


set()

In [346]:
raw_merged_df = pd.merge(df_raw_features, df, on='NEFG2_cleaned', how='outer', suffixes=('_raw', '_pop'))

df_raw_features.to_csv(".././data/analysis/df_clustering_raw.csv")